In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px


In [ ]:
test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
holidays = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')

In [ ]:
# Changing the column to datetime
train['date'] = pd.to_datetime(train['date'])
transactions.date = pd.to_datetime(transactions.date)

In [ ]:
date = []
month =[]
year = []
for i in train['date']:
  date.append(i.strftime('%d'))
  month.append(i.strftime('%m'))
  year.append(i.strftime('%Y'))

train['dd'] = date
train['mm'] = month
train['yy'] = year

In [ ]:
grouped_by_month = train.groupby(['mm','family'])['sales'].sum().reset_index()
grouped_by_year = train.groupby(['yy','family'])['sales'].sum().reset_index()
grouped_by_date = train.groupby(['dd','family'])['sales'].sum().reset_index()

In [ ]:
px.bar(data_frame=grouped_by_month,x='mm',y='sales',color='family')

In [ ]:
px.bar(data_frame=grouped_by_year,x='yy',y='sales',color='family')

In [ ]:
px.bar(data_frame=grouped_by_date,x='dd',y='sales',color='family')

1. Frozen foods sales quadruples during december (why?)

2. Sales of Produce during April and May is half than the other months.

3. Groceries are the highest bought goods in the stores and they stay constant throughout the year

4. Saw a decrease in sales for the year 2017

5. Sales during the last day of every month is lower

In [ ]:
grouped_by_family = train.groupby(['family','date'])['sales'].sum().reset_index()

In [ ]:
argument = []
flag = 0
for i in grouped_by_family['family'].unique():
  argument.append(dict(
                    args=[{'visible': [True if j==flag else False for j in range(grouped_by_family['family'].nunique())]}],
                    label=i,
                    method="restyle"
                ))
  flag +=1

In [ ]:
fig = go.Figure()
for fam in grouped_by_family['family'].unique():
  fig.add_trace(go.Line(x=grouped_by_family[grouped_by_family['family'] == fam]['date'], y=grouped_by_family[grouped_by_family['family'] == fam]['sales'], visible=False))

fig.update_layout(
    updatemenus=[
        dict(
            buttons=list(
                argument
            ),
            direction="down",
        ),
    ]
)

fig.show()

Switch to Frozen foods tab.

In [ ]:
oil.info()

In [ ]:
oil['date'] = pd.to_datetime(oil['date'])

In [ ]:
train_oil = pd.merge(train, oil, how='left', on='date')

In [ ]:
oil_price = train_oil.groupby('date')['dcoilwtico'].mean().reset_index()
input_to_plot = train.groupby('date')['sales'].sum().reset_index()

In [ ]:
px.line(data_frame=oil_price, x='date', y='dcoilwtico')

In [ ]:
normal = (input_to_plot['sales']/min(input_to_plot['sales'])) / (max(input_to_plot['sales'])/min(input_to_plot['sales']))*100
fig = go.Figure()
fig.add_trace(go.Line(x=oil_price['date'], y=oil_price['dcoilwtico'], name='Oil Price'))
fig.add_trace(go.Scatter(x=input_to_plot['date'], y=normal, name='Sales'))